# Model with Embeddings Artefacts for Plug and Play

The plug and play use case provides an opportunity for customers to train their own models and leverage our infrastructure and abstractions to get their models hosted and running on a scalable service with an easy to integrate API. This notebook covers the creation of a model with embeddings, the artefacts that are required by our service, as well as what the resulting API looks like.

Below are the imports that we'll be using to create/train the model, as well as generate the artefacts required:

In [ ]:
import json
import tensorflow as tf
import tensorflow.keras as K
import tensorflow_datasets as tfds
import pandas as pd
import numpy as np

## Creating a Model with Embeddings - Word2Vec

To demonstrate the artefacts required as well as provide some recommendations on the overall structure of the model, we will create a model that generates predictions of embedding components and the result is computed as the minimum distance to elements of an embedding dataset with a configurable distance function. The resulting artefacts can be uploaded to the Abacus.AI platform where it can be hosted as a deployment. The artefacts produced are:
* tensorflow saved model
* embedding dataset
* verification samples (optional)

So first lets get our data using tensorflow_datasets:

In [ ]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

train_batches = train_data.shuffle(1000).padded_batch(10)
test_batches = test_data.shuffle(1000).padded_batch(10)

Next we define the model structure:

In [ ]:
encoder = info.features['text'].encoder
embedding_dim=16

model = K.Sequential([
  K.layers.Embedding(encoder.vocab_size, embedding_dim),
  K.layers.GlobalAveragePooling1D(),
  K.layers.Dense(16, activation='relu'),
  K.layers.Dense(1)
])

model.summary()

And then we kick off the training:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

### Splitting up the model from the embeddings

`model` has now been trained, but for use in Abacus.AI, we need to split the embeddings from the model itself. Below we create the `prediction_model` and `embedding_weights`, which will later be saved/formatted for uploading.

In [ ]:
prediction_model = K.models.clone_model(model)
prediction_model.pop()
prediction_model.pop()
prediction_model.summary()

print()
embedding_weights = model.layers[0].get_weights()[0][1:,:]
print(f'Embedding weights: {embedding_weights.shape}')

### [OPTIONAL] Generate verification data from model and embeddings

An optional artefact that is supported is a verifications file. This contains inputs and the corresponding expected outputs for the model. This file can be used to confirm the correctness of the model served by Abacus.AI. For this example, we are using the cosine distance.

In [ ]:
verification_input = test_batches.unbatch().batch(1).take(10)
num_results = 5
requests = [{
    'input': [[int(x) for x in e[0][0]]],
    'num': num_results,
    'distance': 'cosine'
} for e in list(verification_input.as_numpy_iterator())]

prediction_output = prediction_model.predict(verification_input)

def norm(m):
    return m / np.sqrt(np.sum(m * m, axis=-1, keepdims=1))

scores = norm(prediction_output) @ norm(embedding_weights).T

examples = prediction_output.shape[0]
scored_ix = np.arange(examples).reshape(-1, 1)
top_k = scores.argpartition(-num_results)[:,-num_results:]
sorted_k = top_k[scored_ix, (scores[scored_ix, top_k]).argsort()]
scores_k = scores[scored_ix, sorted_k]

responses = [
    {'result': [{'term': encoder.decode([i + 1]).rstrip(), 'score': float(s)}
                for i, s in zip(terms, scores)]}
    for terms, scores in zip(top_k, scores_k)]

### [IMPORTANT] Update the layers of the created model

When the model is hosted, our api accepts multiple types of inputs and needs to be able to determine what gets passed on to the model. To resolve this, we inspect the model to discover its input tensor(s). Below we add a `K.layers.InputLayer` in order to name the input tensor as `tokens`. As a result of this, the prediction api will look for the `tokens` parameter and take whatever the value is, convert it into a tensor and pass it on to the model.

With the below definition, lets exampine this example curl request:
```bash
curl --globoff "http://abacus.ai/api/v0/predict?deploymentToken=foobar&deploymentId=baz&notSent=deadbeef&tokens=[[123,456]]"
```
Of all the query parameters, only `tokens=[[123,456]]` will be converted into a tensor to be passed into the model. The `deploymentToken` and `deploymentId` are required parameters for our API and the `notSent=deadbeef` will be dropped.

In [ ]:
model_to_save = K.Sequential([
    K.layers.InputLayer(input_shape=(None,), name='tokens'),
    prediction_model,
    K.layers.Lambda(lambda x: tf.reduce_mean(x, axis=0), name='result')
])
model_to_save.summary()

With Keras models, the `InputLayer` is actually hidden. So to confirm that we have successfully named the input tensor we can check with the following:

In [ ]:
model_to_save.input_names

### Write out all Artefacts

Now its time to generate the various required artefacts. For the model, we use the TensorFlow SavedModel format and compress that into a tarball. Then, for the embeddings, we use pandas to write it out as a csv. Finally we also create the optional verifications file, which is in a jsonl format. In the end we have 3 artifacts as well as the folder where the SavedModel is saved.

In [ ]:
!mkdir -p /tmp/word2vec/model
saved_model_dir = '/tmp/word2vec/model'
model_to_save.save(saved_model_dir)

!tar -cvzf /tmp/word2vec/model.tgz -C /tmp/word2vec/model .

pd.DataFrame(
    embedding_weights,
    index=pd.Index(
        [encoder.decode([i]).rstrip() for i in range(1, encoder.vocab_size)],
        name='term')
).to_csv('/tmp/word2vec/embedding.csv')

# Creating the optional verification file
with open('/tmp/word2vec/verification.jsonl', 'wt') as f:
    for req, resp in zip(requests, responses):
        json.dump({'request': req, 'response': resp}, f)
        f.write('\n')

!ls -l /tmp/word2vec

### [RECOMMENDED] Verify saved model

Abacus.AI currently does not support the definition of custom objects and it is possible there may be other problems encountered when loading the model. A good check is to load the model that we created earlier from disk:

In [ ]:
model_from_disk = tf.keras.models.load_model(saved_model_dir)
model_from_disk.summary()

Upon loading the model, we can also inspect the structure of the input tensor. This is useful to confirm that the InputLayer was correctly set in the model that was saved. The following is code similar to that used within Abacus.AI to discover the name of the input tensor:

In [ ]:
print('Input Tensors: ', [tensor for tensor in model_from_disk.signatures['serving_default'].structured_input_signature if tensor]) # Cleanup empty inputs